In [ ]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 1.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import io
import gc
import numpy as np
import sklearn
from scipy import stats
from category_encoders import MEstimateEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import scipy.stats as st
from scipy.stats import f_oneway
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving hotel_booking.csv to hotel_booking.csv


In [ ]:
# df = pd.read_csv(io.BytesIO(uploaded['hotel_booking.csv']))
df = pd.read_csv('hotel_booking.csv')
RAW_DF_NUM_ROWS = df.shape[0]

print(df.shape)
df.head()

FileNotFoundError: ignored

In [ ]:
df.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

In [ ]:
# 20 numerical cols + 16 discrete cols = total 36 cols
len(df.select_dtypes("number").columns), len(df.select_dtypes(["object", "category"]).columns)

(20, 16)

In [ ]:
# lists all numeric: int or float cols
numeric_cols = df.select_dtypes("number").columns.tolist()
numeric_cols

['is_canceled',
 'lead_time',
 'arrival_date_year',
 'arrival_date_week_number',
 'arrival_date_day_of_month',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'is_repeated_guest',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'booking_changes',
 'agent',
 'company',
 'days_in_waiting_list',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests']

In [ ]:
categorical_cols = df.select_dtypes(["object", "category"]).columns.tolist()
categorical_cols

['hotel',
 'arrival_date_month',
 'meal',
 'country',
 'market_segment',
 'distribution_channel',
 'reserved_room_type',
 'assigned_room_type',
 'deposit_type',
 'customer_type',
 'reservation_status',
 'reservation_status_date',
 'name',
 'email',
 'phone-number',
 'credit_card']

In [ ]:
print("cardinality of column is_canceled =", df['is_canceled'].nunique(), "\n")

print("cardinality of all numeric columns:")
df[numeric_cols].apply(pd.Series.nunique)

cardinality of column is_canceled = 2 

cardinality of all numeric columns:


is_canceled                          2
lead_time                          479
arrival_date_year                    3
arrival_date_week_number            53
arrival_date_day_of_month           31
stays_in_weekend_nights             17
stays_in_week_nights                35
adults                              14
children                             5
babies                               5
is_repeated_guest                    2
previous_cancellations              15
previous_bookings_not_canceled      73
booking_changes                     21
agent                              333
company                            352
days_in_waiting_list               128
adr                               8879
required_car_parking_spaces          5
total_of_special_requests            6
dtype: int64

In [ ]:
temp_cat_cols = ["is_canceled", "arrival_date_year", "is_repeated_guest", "agent", "company"]

# update categorical_cols and numeric_cols
categorical_cols = categorical_cols + temp_cat_cols
numeric_cols = list(set(numeric_cols) - set(temp_cat_cols))
del temp_cat_cols
gc.collect()

# make sure they sum up to toal no. of columns in df
assert(len(categorical_cols) + len(numeric_cols) == df.shape[1])
len(categorical_cols), len(numeric_cols)

(21, 15)

In [ ]:
# describe() table's : column count also is 20 --> same as number of numeric cols
# i.e it describes only numeric cols
print(df.describe().shape)

df.describe()

(8, 20)


,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
count,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119386.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,103050.000000,6797.000000,119390.000000,119390.000000,119390.000000,119390.000000
mean,0.370416,104.011416,2016.156554,27.165173,15.798241,0.927599,2.500302,1.856403,0.103890,0.007949,0.031912,0.087118,0.137097,0.221124,86.693382,189.266735,2.321149,101.831122,0.062518,0.571363
std,0.482918,106.863097,0.707476,13.605138,8.780829,0.998613,1.908286,0.579261,0.398561,0.097436,0.175767,0.844336,1.497437,0.652306,110.774548,131.655015,17.594721,50.535790,0.245291,0.792798
min,0.000000,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000,0.000000,-6.380000,0.000000,0.000000
25%,0.000000,18.000000,2016.000000,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,62.000000,0.000000,69.290000,0.000000,0.000000
50%,0.000000,69.000000,2016.000000,28.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,179.000000,0.000000,94.575000,0.000000,0.000000
75%,1.000000,160.000000,2017.000000,38.000000,23.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,229.000000,270.000000,0.000000,126.000000,0.000000,1.000000
max,1.000000,737.000000,2017.000000,53.000000,31.000000,19.000000,50.000000,55.000000,10.000000,10.000000,1.000000,26.000000,72.000000,21.000000,535.000000,543.000000,391.000000,5400.000000,8.000000,5.000000


In [ ]:
print("len(categorical_cols) =", len(categorical_cols))

# is_canceled is the label/target col
# hence remove is_canceled from categorical_cols
categorical_cols.remove('is_canceled')
print("now len(categorical_cols) =", len(categorical_cols))

len(categorical_cols) = 21
now len(categorical_cols) = 20


In [ ]:
# demonstraion of mean-target-encoding:
# label or target column is is_canceled
df["hotel_encoded"] = df.groupby("hotel")["is_canceled"].transform("mean")

# for this column there is a big differenece between their mean-target-encoded values.
# this tells us that this column is important.
df[["hotel", "hotel_encoded"]].drop_duplicates()

,hotel,hotel_encoded
0,Resort Hotel,0.277634
40060,City Hotel,0.417270


In [ ]:
# remove the encoded column hotel_encoded which was just for a demo.
df = df.drop(columns=["hotel_encoded"])
df.shape

(119390, 36)

In [ ]:
# Create the encoder instance. Choose m to control noise.
encoder = MEstimateEncoder(cols=categorical_cols, m=5.0)

# Fit the encoder on the encoding split.
encoder.fit(df, df["is_canceled"])

# Encode the Zipcode column to create the final training data
df = encoder.transform(df)
print(df.shape)
df.head()

(119390, 36)


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,email,phone-number,credit_card
0,0.277645,0,342,0.370158,0.374534,27,1,0,0,2,...,0.407461,0.0,0,0,0.000025,0.692604,0.308680,0.30868,0.370416,0.316146
1,0.277645,0,737,0.370158,0.374534,27,1,0,0,2,...,0.407461,0.0,0,0,0.000025,0.692604,0.264583,0.30868,0.370416,0.365755
2,0.277645,0,7,0.370158,0.374534,27,1,0,1,1,...,0.407461,75.0,0,0,0.000025,0.978591,0.231510,0.30868,0.370416,0.292604
3,0.277645,0,13,0.370158,0.374534,27,1,0,1,1,...,0.407461,75.0,0,0,0.000025,0.978591,0.308680,0.30868,0.370416,0.365755
4,0.277645,0,14,0.370158,0.374534,27,1,0,2,2,...,0.407461,98.0,0,1,0.000025,0.518051,0.308680,0.30868,0.370416,0.602264


In [ ]:
# all cols are now numeric !
df.dtypes

hotel                             float64
is_canceled                         int64
lead_time                           int64
arrival_date_year                 float64
arrival_date_month                float64
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                              float64
country                           float64
market_segment                    float64
distribution_channel              float64
is_repeated_guest                 float64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                float64
assigned_room_type                float64
booking_changes                     int64
deposit_type                      float64
agent                             

# Clean features

In [ ]:
# create features/Independent Vars (IVs) and labels/target/Dependent Vars (DVs)
X = df.drop(columns="is_canceled")
x_cols = X.columns
y = df["is_canceled"]
X.shape, y.shape

((119390, 35), (119390,))

In [ ]:
corr = X.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

<ipython-input-80-c64628ba55fb>:2: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  corr.style.background_gradient(cmap='coolwarm').set_precision(2)
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/style.py:3931: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/style.py:3932: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,email,phone-number,credit_card
hotel,1.00,0.08,0.00,0.05,0.00,-0.00,-0.19,-0.23,-0.01,-0.04,-0.04,0.06,-0.03,0.10,0.18,0.05,-0.01,-0.00,0.20,0.28,-0.07,0.17,0.28,0.07,0.07,-0.01,0.10,-0.22,-0.04,0.14,0.11,0.10,0.13,nan,0.04
lead_time,0.08,1.00,0.03,0.22,0.13,0.00,0.09,0.17,0.12,-0.04,-0.02,-0.04,0.05,0.38,0.22,0.12,0.09,-0.07,0.09,0.17,0.00,0.38,0.31,0.10,0.17,-0.16,-0.06,-0.12,-0.10,0.29,0.25,0.22,0.29,nan,0.08
arrival_date_year,0.00,0.03,1.00,0.03,-0.33,-0.01,0.04,0.04,0.02,0.03,-0.01,-0.02,-0.06,0.00,0.00,-0.01,-0.02,0.01,-0.04,0.00,0.01,-0.03,-0.05,0.01,-0.09,0.05,0.15,-0.02,0.07,0.03,-0.08,0.02,0.03,nan,0.01
arrival_date_month,0.05,0.22,0.03,1.00,0.15,0.00,0.05,0.04,0.09,0.02,-0.00,-0.01,-0.05,0.05,0.08,0.06,0.01,-0.02,-0.04,0.02,-0.01,0.06,0.07,0.05,0.03,-0.03,0.28,-0.02,0.00,0.07,-0.09,0.06,0.07,nan,0.01
arrival_date_week_number,0.00,0.13,-0.33,0.15,1.00,0.07,0.02,0.02,0.03,0.01,0.01,0.02,0.02,0.01,0.01,0.03,0.04,-0.02,0.01,0.01,0.01,0.01,0.03,-0.01,0.02,-0.07,0.08,0.00,0.03,0.01,-0.11,0.01,0.01,nan,0.00
arrival_date_day_of_month,-0.00,0.00,-0.01,0.00,0.07,1.00,-0.02,-0.03,-0.00,0.01,-0.00,-0.01,0.00,0.01,-0.00,0.01,-0.03,-0.00,-0.02,-0.01,0.01,-0.01,-0.01,0.01,0.02,-0.00,0.03,0.01,0.00,-0.01,-0.05,-0.00,-0.01,nan,-0.00
stays_in_weekend_nights,-0.19,0.09,0.04,0.05,0.02,-0.02,1.00,0.50,0.09,0.05,0.02,-0.04,-0.14,0.00,0.08,0.09,-0.01,-0.04,-0.15,-0.08,0.06,-0.11,-0.12,0.06,-0.05,0.04,0.05,-0.02,0.07,-0.00,0.01,0.00,-0.00,nan,-0.00
stays_in_week_nights,-0.23,0.17,0.04,0.04,0.02,-0.03,0.50,1.00,0.09,0.04,0.02,-0.05,-0.14,-0.01,0.07,0.10,-0.01,-0.05,-0.17,-0.10,0.10,-0.08,-0.12,0.06,-0.00,0.03,0.07,-0.02,0.07,0.02,-0.09,0.02,0.02,nan,0.01
adults,-0.01,0.12,0.02,0.09,0.03,-0.00,0.09,0.09,1.00,0.03,0.02,-0.01,-0.11,0.04,0.13,0.15,-0.01,-0.11,-0.18,-0.13,-0.05,-0.03,0.01,0.17,-0.01,0.09,0.23,0.01,0.12,0.06,-0.02,0.04,0.06,nan,0.02
children,-0.04,-0.04,0.03,0.02,0.01,0.01,0.05,0.04,0.03,1.00,0.02,0.00,-0.05,-0.09,-0.01,0.03,-0.02,-0.02,-0.17,-0.20,0.05,-0.10,-0.05,0.04,-0.03,0.10,0.32,0.06,0.08,0.01,-0.07,0.00,0.00,nan,-0.00


In [ ]:
removed_cols = ["babies", "is_repeated_guest", "required_car_parking_spaces", "phone-number"]
df = df.drop(columns=removed_cols)

# create features/Independent Vars (IVs) and labels/target/Dependent Vars (DVs)
X = df.drop(columns="is_canceled")
x_cols = X.columns
y = df["is_canceled"]
print(X.shape, y.shape)

corr = X.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

(119390, 31) (119390,)


<ipython-input-81-6ca236dd2f82>:11: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  corr.style.background_gradient(cmap='coolwarm').set_precision(2)


,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,meal,country,market_segment,distribution_channel,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,total_of_special_requests,reservation_status,reservation_status_date,name,email,credit_card
hotel,1.00,0.08,0.00,0.05,0.00,-0.00,-0.19,-0.23,-0.01,-0.04,0.06,-0.03,0.10,0.18,-0.01,-0.00,0.20,0.28,-0.07,0.17,0.28,0.07,0.07,-0.01,0.10,-0.04,0.14,0.11,0.10,0.13,0.04
lead_time,0.08,1.00,0.03,0.22,0.13,0.00,0.09,0.17,0.12,-0.04,-0.04,0.05,0.38,0.22,0.09,-0.07,0.09,0.17,0.00,0.38,0.31,0.10,0.17,-0.16,-0.06,-0.10,0.29,0.25,0.22,0.29,0.08
arrival_date_year,0.00,0.03,1.00,0.03,-0.33,-0.01,0.04,0.04,0.02,0.03,-0.02,-0.06,0.00,0.00,-0.02,0.01,-0.04,0.00,0.01,-0.03,-0.05,0.01,-0.09,0.05,0.15,0.07,0.03,-0.08,0.02,0.03,0.01
arrival_date_month,0.05,0.22,0.03,1.00,0.15,0.00,0.05,0.04,0.09,0.02,-0.01,-0.05,0.05,0.08,0.01,-0.02,-0.04,0.02,-0.01,0.06,0.07,0.05,0.03,-0.03,0.28,0.00,0.07,-0.09,0.06,0.07,0.01
arrival_date_week_number,0.00,0.13,-0.33,0.15,1.00,0.07,0.02,0.02,0.03,0.01,0.02,0.02,0.01,0.01,0.04,-0.02,0.01,0.01,0.01,0.01,0.03,-0.01,0.02,-0.07,0.08,0.03,0.01,-0.11,0.01,0.01,0.00
arrival_date_day_of_month,-0.00,0.00,-0.01,0.00,0.07,1.00,-0.02,-0.03,-0.00,0.01,-0.01,0.00,0.01,-0.00,-0.03,-0.00,-0.02,-0.01,0.01,-0.01,-0.01,0.01,0.02,-0.00,0.03,0.00,-0.01,-0.05,-0.00,-0.01,-0.00
stays_in_weekend_nights,-0.19,0.09,0.04,0.05,0.02,-0.02,1.00,0.50,0.09,0.05,-0.04,-0.14,0.00,0.08,-0.01,-0.04,-0.15,-0.08,0.06,-0.11,-0.12,0.06,-0.05,0.04,0.05,0.07,-0.00,0.01,0.00,-0.00,-0.00
stays_in_week_nights,-0.23,0.17,0.04,0.04,0.02,-0.03,0.50,1.00,0.09,0.04,-0.05,-0.14,-0.01,0.07,-0.01,-0.05,-0.17,-0.10,0.10,-0.08,-0.12,0.06,-0.00,0.03,0.07,0.07,0.02,-0.09,0.02,0.02,0.01
adults,-0.01,0.12,0.02,0.09,0.03,-0.00,0.09,0.09,1.00,0.03,-0.01,-0.11,0.04,0.13,-0.01,-0.11,-0.18,-0.13,-0.05,-0.03,0.01,0.17,-0.01,0.09,0.23,0.12,0.06,-0.02,0.04,0.06,0.02
children,-0.04,-0.04,0.03,0.02,0.01,0.01,0.05,0.04,0.03,1.00,0.00,-0.05,-0.09,-0.01,-0.02,-0.02,-0.17,-0.20,0.05,-0.10,-0.05,0.04,-0.03,0.10,0.32,0.08,0.01,-0.07,0.00,0.00,-0.00


# Standardize features
Shift and scale each column so that:

i. mean of each column = 0

ii. stdev of each column = 1

In [ ]:
# this returns a scaled numpy array
scaler = StandardScaler()
X = scaler.fit_transform(X)

print("X.shape=", X.shape, "y.shape=", y.shape)
X

X.shape= (119390, 31) y.shape= (119390,)


array([[-1.40722407,  2.22705112, -0.02042056, ..., -0.67048972,
        -0.75537854, -0.57444971],
       [-1.40722407,  5.9233847 , -0.02042056, ..., -1.15339497,
        -0.75537854, -0.0481631 ],
       [-1.40722407, -0.90781407, -0.02042056, ..., -1.5155739 ,
        -0.75537854, -0.82419467],
       ...,
       [ 0.71061889, -0.65515329,  1.31164902, ...,  0.63627214,
        -0.75537854, -0.59633008],
       [ 0.71061889,  0.0466822 ,  1.31164902, ..., -0.67048972,
        -0.75537854, -0.29376352],
       [ 0.71061889,  0.94503163,  1.31164902, ..., -0.67048972,
        -0.75537854,  0.90761227]])

# Univariate outlier treatment

## Quantile outlier elimination
q_low = df["col"].quantile(0.01)

q_hi  = df["col"].quantile(0.99)

df_filtered = df[(df["col"] < q_hi) & (df["col"] > q_low)]

## Z-score: remove all rows that have outliers in, at least, one column

In [ ]:
# all cols are already scaled to have stdev=1
# hence, a z-score of 3 for each col is at value 3 only
indexes = (np.abs(X) < 3).all(axis=1)
print(indexes)

X = X[indexes]
y = y[indexes]

print("X.shape=", X.shape, "y.shape=", y.shape)

[False False  True ... False  True  True]


((96466, 31), (96466,))

## Or instead of removing, just clip values between [-3, +3] stdev

In [ ]:
# df = df.clip(-3, 3)
X = np.clip(X, a_min=-3, a_max=3)
print("X.shape=", X.shape, "y.shape=", y.shape)
X

(96466, 31) (96466,)


array([[-1.40722407, -0.90781407, -0.02042056, ..., -1.5155739 ,
        -0.75537854, -0.82419467],
       [-1.40722407, -0.85166723, -0.02042056, ..., -0.67048972,
        -0.75537854, -0.0481631 ],
       [-1.40722407, -0.84230942, -0.02042056, ..., -0.67048972,
        -0.75537854,  2.46087512],
       ...,
       [ 0.71061889, -0.01882245,  1.31164902, ..., -0.67048972,
        -0.75537854,  0.45587756],
       [ 0.71061889,  0.0466822 ,  1.31164902, ..., -0.67048972,
        -0.75537854, -0.29376352],
       [ 0.71061889,  0.94503163,  1.31164902, ..., -0.67048972,
        -0.75537854,  0.90761227]])

# Multivariate Outliers: Remove outliers through Mahalanobis distance
E.g:

i) a day with high snowing is not an outlier

ii) a day with high sunshine is not an outlier

iii) but a day with both high snowing and high sunshine is an outlier because its joint probability is extremely low.

Such an outlier cannot be detected by uni-variate treatment. We need multi-varite treatment to detect such outliers e.g. Mahalanobis distance.


In [ ]:
# function to calculate the Mahalanobis distance
def calculateMahalanobis(y=None, data=None, cov=None):

    y_mu = y - np.mean(data, axis=0)
    if not cov:
        cov = np.cov(data.values.T)
    inv_covmat = np.linalg.inv(cov)
    mahalanobis = np.einsum('ij,jk,ki->i', y_mu, inv_covmat, y_mu.T)
    return mahalanobis

X = pd.DataFrame(X, columns=x_cols)

# drop rows with at least 1 null value
X['is_canceled'] = y
X = X.dropna()
y = X['is_canceled']
X = X.drop(columns="is_canceled")

X['mahalanobis'] = calculateMahalanobis(y=X, data=X)

# calculate p-value for each mahalanobis distance
X['p'] = 1 - chi2.cdf(X['mahalanobis'], X.shape[1]-1)

print("X.shape=", X.shape, "y.shape=", y.shape)
X.head()

X.shape= (119386, 33) y.shape= (119386,)


,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,customer_type,adr,total_of_special_requests,reservation_status,reservation_status_date,name,email,credit_card,mahalanobis,p
0,-1.407224,2.227051,-0.020421,0.121415,-0.012141,-1.685297,-0.92889,-1.310240,0.247897,-0.260663,...,0.562464,-2.015038,-0.720694,-0.76704,1.419422,-0.670490,-0.755379,-0.574450,59.270513,1.643535e-03
1,-1.407224,5.923385,-0.020421,0.121415,-0.012141,-1.685297,-0.92889,-1.310240,0.247897,-0.260663,...,0.562464,-2.015038,-0.720694,-0.76704,1.419422,-1.153395,-0.755379,-0.048163,127.784638,1.035838e-13
2,-1.407224,-0.907814,-0.020421,0.121415,-0.012141,-1.685297,-0.92889,-0.786207,-1.478447,-0.260663,...,0.562464,-0.530935,-0.720694,-0.76704,2.679717,-1.515574,-0.755379,-0.824195,48.346250,2.437273e-02
3,-1.407224,-0.851667,-0.020421,0.121415,-0.012141,-1.685297,-0.92889,-0.786207,-1.478447,-0.260663,...,0.562464,-0.530935,-0.720694,-0.76704,2.679717,-0.670490,-0.755379,-0.048163,35.975054,2.467970e-01
4,-1.407224,-0.842309,-0.020421,0.121415,-0.012141,-1.685297,-0.92889,-0.262174,0.247897,-0.260663,...,0.562464,-0.075810,0.540666,-0.76704,0.650198,-0.670490,-0.755379,2.460875,22.473416,8.675848e-01


In [ ]:
# drop rows where p-value <= 0.1
X['is_canceled'] = y
X = X[X.p > 0.1]
y = X['is_canceled']

# drop the cols which are not required now
X = X.drop(columns=["is_canceled", "p", "mahalanobis"])
print("X.shape=", X.shape, "y.shape=", y.shape)
X.head()

X.shape= (103245, 31) y.shape= (103245,)


,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,company,days_in_waiting_list,customer_type,adr,total_of_special_requests,reservation_status,reservation_status_date,name,email,credit_card
3,-1.407224,-0.851667,-0.020421,0.121415,-0.012141,-1.685297,-0.92889,-0.786207,-1.478447,-0.260663,...,0.173255,-0.131924,0.562464,-0.530935,-0.720694,-0.767040,2.679717,-0.670490,-0.755379,-0.048163
4,-1.407224,-0.842309,-0.020421,0.121415,-0.012141,-1.685297,-0.92889,-0.262174,0.247897,-0.260663,...,0.173255,-0.131924,0.562464,-0.075810,0.540666,-0.767040,0.650198,-0.670490,-0.755379,2.460875
5,-1.407224,-0.842309,-0.020421,0.121415,-0.012141,-1.685297,-0.92889,-0.262174,0.247897,-0.260663,...,0.173255,-0.131924,0.562464,-0.075810,0.540666,-0.767040,0.650198,-0.670490,-0.755379,1.553852
6,-1.407224,-0.973319,-0.020421,0.121415,-0.012141,-1.685297,-0.92889,-0.262174,0.247897,-0.260663,...,0.173255,-0.131924,0.562464,0.102282,-0.720694,-0.767040,0.650198,-0.670490,-0.755379,-1.911211
8,-1.407224,-0.177905,-0.020421,0.121415,-0.012141,-1.685297,-0.92889,0.261858,0.247897,-0.260663,...,0.173255,-0.131924,0.562464,-0.392419,0.540666,1.303855,1.512942,0.636272,1.286372,1.310607


# Notice the row-count has reduced in the final dataframe from the raw one..
The outlier rows are dropped.

In [ ]:
dropped_num_rows = RAW_DF_NUM_ROWS - X.shape[0]
dropped_num_rows

16145

# Note: If you do multivariate outlier treatment, then you need NOT do univariate outlier treatment.

In [ ]:
X.head(4)

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,company,days_in_waiting_list,customer_type,adr,total_of_special_requests,reservation_status,reservation_status_date,name,email,credit_card
3,-1.407224,-0.851667,-0.020421,0.121415,-0.012141,-1.685297,-0.92889,-0.786207,-1.478447,-0.260663,...,0.173255,-0.131924,0.562464,-0.530935,-0.720694,-0.76704,2.679717,-0.67049,-0.755379,-0.048163
4,-1.407224,-0.842309,-0.020421,0.121415,-0.012141,-1.685297,-0.92889,-0.262174,0.247897,-0.260663,...,0.173255,-0.131924,0.562464,-0.075810,0.540666,-0.76704,0.650198,-0.67049,-0.755379,2.460875
5,-1.407224,-0.842309,-0.020421,0.121415,-0.012141,-1.685297,-0.92889,-0.262174,0.247897,-0.260663,...,0.173255,-0.131924,0.562464,-0.075810,0.540666,-0.76704,0.650198,-0.67049,-0.755379,1.553852
6,-1.407224,-0.973319,-0.020421,0.121415,-0.012141,-1.685297,-0.92889,-0.262174,0.247897,-0.260663,...,0.173255,-0.131924,0.562464,0.102282,-0.720694,-0.76704,0.650198,-0.67049,-0.755379,-1.911211


In [ ]:
print("X.shape=", X.shape, "y.shape=", y.shape)
y

X.shape= (103245, 31) y.shape= (103245,)


3         0
4         0
5         0
6         0
8         1
         ..
119385    0
119386    0
119387    0
119388    0
119389    0
Name: is_canceled, Length: 103245, dtype: int64

In [ ]:
X = X.reset_index()
y = y.reset_index()

# 1-way ANOVA: find important continuous cols when label is categorical and vice-versa.

In [ ]:
# X can be divided into two sets of rows where y==0 and where y==1
assert (X.loc[y==0].shape[0] + X.loc[y==1].shape[0] == X.shape[0])

# similarly this applies to any 1 column or list of columns
assert (X["hotel"].loc[y==0].shape[0] + X["hotel"].loc[y==1].shape[0] == X.shape[0])

In [ ]:
# DEMO: print ANOVA p-values for two columns

for ccol in X.columns[:2]:
  # ccol : stands for continuous_column_name.
  # after mean-target-encoding all are continuous cols

  zero_group_vals = X[ccol].loc[y==0].values
  ones_group_vals = X[ccol].loc[y==1].values

  print("zero_group_vals =", zero_group_vals)
  print("ones_group_vals =", ones_group_vals)

  p_value = f_oneway(zero_group_vals, ones_group_vals).pvalue
  print(f"\n ------- Column {ccol} ANOVA p_value = {p_value} --------- \n")

zero_group_vals = [-1.40722407 -1.40722407 -1.40722407 ...  0.71061889  0.71061889
  0.71061889]
ones_group_vals = [-1.40722407 -1.40722407 -1.40722407 ...  0.71061889  0.71061889
  0.71061889]

 ------- Column hotel ANOVA p_value = 0.0 --------- 

zero_group_vals = [-0.85166723 -0.84230942 -0.84230942 ... -0.65515329  0.0466822
  0.94503163]
ones_group_vals = [-0.17790516 -0.27148322 -0.75808916 ... -0.73937355 -0.93588749
 -0.90781407]

 ------- Column lead_time ANOVA p_value = 0.0 --------- 



In [ ]:
# get ANOVA p-values for all the columns and select cols where p-value < 0.01
print("X.shape=", X.shape, "y.shape=", y.shape)

p_values = []
for ccol in X.columns:
  # ccol : stands for continuous_column_name.
  # after mean-target-encoding all are continuous cols

  zero_group_vals = X[ccol].loc[y==0].values
  ones_group_vals = X[ccol].loc[y==1].values

  p_value = f_oneway(zero_group_vals, ones_group_vals).pvalue
  p_values.append(p_value)

p_values = np.array(p_values)
corr_cols = list(X.columns[p_values < 0.01])
print("num columns p-value < 0.01 =", len(corr_cols))

print("columns which are correlated with target label =", corr_cols)
print("columns which are independent of target label =", set(X.columns)-set(corr_cols) )

# select correlated columns only: ANOVA feature selection done
X = X[corr_cols]

print("X.shape=", X.shape, "y.shape=", y.shape)

X.shape= (103245, 31) y.shape= (103245,)
num columns p-value < 0.01 = 29
columns which are correlated with target label = ['hotel', 'lead_time', 'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'meal', 'country', 'market_segment', 'distribution_channel', 'previous_cancellations', 'previous_bookings_not_canceled', 'reserved_room_type', 'assigned_room_type', 'booking_changes', 'deposit_type', 'agent', 'company', 'days_in_waiting_list', 'customer_type', 'adr', 'total_of_special_requests', 'reservation_status', 'reservation_status_date', 'name', 'email', 'credit_card']
columns which are independent of target label = {'children', 'arrival_date_day_of_month'}
X.shape= (103245, 29) y.shape= (103245,)


### ----- Done demo of 1-way ANOVA feature selection ------

# $Chi^2$ feature selection: find important categorical cols when label is also categorical.

1. Remove high cardinality categorical columns otherwise OutOfMemoryError during one-hot encoding.

2. Label-encode and One-Hot-encode categorical columns.

3. Remove sparse columns i.e cols which have values in < 5% rows.

4. Do $Chi^2$ test.

In [ ]:
df = pd.read_csv('hotel_booking.csv')

# drop columns which have nan values
df = df.dropna(axis=1)

numeric_cols = df.select_dtypes("number").columns.tolist()
categorical_cols = df.select_dtypes(["object", "category"]).columns.tolist()
temp_cat_cols = ["is_canceled", "arrival_date_year", "is_repeated_guest"]
categorical_cols = categorical_cols + temp_cat_cols
numeric_cols = list(set(numeric_cols) - set(temp_cat_cols))
del temp_cat_cols

# column is_canceled is Label
y = df["is_canceled"]
categorical_cols.remove('is_canceled')
df = df.drop(columns="is_canceled")

print("cardinality of y =", y.nunique())
print("df.shape =", df.shape)
print("cardinality of all categorical columns:")
df[categorical_cols].apply(pd.Series.nunique)

df.shape = (119390, 32)
cardinality of all categorical columns:


hotel                           2
arrival_date_month             12
meal                            5
market_segment                  8
distribution_channel            5
reserved_room_type             10
assigned_room_type             12
deposit_type                    3
customer_type                   4
reservation_status              3
reservation_status_date       926
name                        81503
email                      115889
phone-number               119390
credit_card                  9000
arrival_date_year               3
is_repeated_guest               2
dtype: int64

In [ ]:
# find columns with cardinality > 900
categorical_cols = np.array(categorical_cols)
categorical_cols[ (df[categorical_cols].apply(pd.Series.nunique) > 900).values ]

array(['reservation_status_date', 'name', 'email', 'phone-number',
       'credit_card'], dtype='<U23')

### Generally any kind of personal ID should not be a feature e.g Aadhar no. Social Security no., Credit card no., etc.

In [ ]:
 # remove high cardinality columns (say greater than 900) otherwise OutOfMemoryError during one-hot encoding
 print("Originally #columns:", df.shape)

 categorical_cols = list(categorical_cols[ (df[categorical_cols].apply(pd.Series.nunique) < 900).values ])
 df = df[categorical_cols + numeric_cols]

 print("Now after removing high cardinality cols, #columns:", df.shape)
 df.head(4)

Originally #columns: (119390, 32)
Now after removing high cardinality cols, #columns: (119390, 26)


,hotel,arrival_date_month,meal,market_segment,distribution_channel,reserved_room_type,assigned_room_type,deposit_type,customer_type,reservation_status,...,arrival_date_day_of_month,adr,stays_in_week_nights,previous_bookings_not_canceled,days_in_waiting_list,total_of_special_requests,adults,arrival_date_week_number,babies,lead_time
0,Resort Hotel,July,BB,Direct,Direct,C,C,No Deposit,Transient,Check-Out,...,1,0.0,0,0,0,0,2,27,0,342
1,Resort Hotel,July,BB,Direct,Direct,C,C,No Deposit,Transient,Check-Out,...,1,0.0,0,0,0,0,2,27,0,737
2,Resort Hotel,July,BB,Direct,Direct,A,C,No Deposit,Transient,Check-Out,...,1,75.0,1,0,0,0,1,27,0,7
3,Resort Hotel,July,BB,Corporate,Corporate,A,A,No Deposit,Transient,Check-Out,...,1,75.0,1,0,0,0,1,27,0,13


In [ ]:
# One-Hot-Encode categorical cols which then becomes binary cols
df = pd.get_dummies(df, columns=categorical_cols)

print("One-Hot encoded #columns:", df.shape)
print("One-Hot encoded df.columns =", df.columns)
df.head(4)

One-Hot encoded #columns: (119390, 83)
One-Hot encoded df.columns = Index(['stays_in_weekend_nights', 'booking_changes',
       'required_car_parking_spaces', 'previous_cancellations',
       'arrival_date_day_of_month', 'adr', 'stays_in_week_nights',
       'previous_bookings_not_canceled', 'days_in_waiting_list',
       'total_of_special_requests', 'adults', 'arrival_date_week_number',
       'babies', 'lead_time', 'hotel_City Hotel', 'hotel_Resort Hotel',
       'arrival_date_month_April', 'arrival_date_month_August',
       'arrival_date_month_December', 'arrival_date_month_February',
       'arrival_date_month_January', 'arrival_date_month_July',
       'arrival_date_month_June', 'arrival_date_month_March',
       'arrival_date_month_May', 'arrival_date_month_November',
       'arrival_date_month_October', 'arrival_date_month_September', 'meal_BB',
       'meal_FB', 'meal_HB', 'meal_SC', 'meal_Undefined',
       'market_segment_Aviation', 'market_segment_Complementary',
       'ma

,stays_in_weekend_nights,booking_changes,required_car_parking_spaces,previous_cancellations,arrival_date_day_of_month,adr,stays_in_week_nights,previous_bookings_not_canceled,days_in_waiting_list,total_of_special_requests,...,customer_type_Transient,customer_type_Transient-Party,reservation_status_Canceled,reservation_status_Check-Out,reservation_status_No-Show,arrival_date_year_2015,arrival_date_year_2016,arrival_date_year_2017,is_repeated_guest_0,is_repeated_guest_1
0,0,3,0,0,1,0.0,0,0,0,0,...,1,0,0,1,0,1,0,0,1,0
1,0,4,0,0,1,0.0,0,0,0,0,...,1,0,0,1,0,1,0,0,1,0
2,0,0,0,0,1,75.0,1,0,0,0,...,1,0,0,1,0,1,0,0,1,0
3,0,0,0,0,1,75.0,1,0,0,0,...,1,0,0,1,0,1,0,0,1,0


In [ ]:
# drop columns which are constant i.e cardinality==1
constant_cols = df.columns[df.apply(pd.Series.nunique) == 1]
print("no. of constant_cols =", len(constant_cols))

df = df.drop(columns=constant_cols)
print("df.shape =", df.shape)

no. of constant_cols = 0
df.shape = (119390, 83)


### Now there are only continuous and binary columns now

In [ ]:
binary_cols = df.columns[df.apply(pd.Series.nunique) == 2]
print("no. of all binary_cols =", len(binary_cols))
binary_cols

no. of all binary_cols = 69


Index(['hotel_City Hotel', 'hotel_Resort Hotel', 'arrival_date_month_April',
       'arrival_date_month_August', 'arrival_date_month_December',
       'arrival_date_month_February', 'arrival_date_month_January',
       'arrival_date_month_July', 'arrival_date_month_June',
       'arrival_date_month_March', 'arrival_date_month_May',
       'arrival_date_month_November', 'arrival_date_month_October',
       'arrival_date_month_September', 'meal_BB', 'meal_FB', 'meal_HB',
       'meal_SC', 'meal_Undefined', 'market_segment_Aviation',
       'market_segment_Complementary', 'market_segment_Corporate',
       'market_segment_Direct', 'market_segment_Groups',
       'market_segment_Offline TA/TO', 'market_segment_Online TA',
       'market_segment_Undefined', 'distribution_channel_Corporate',
       'distribution_channel_Direct', 'distribution_channel_GDS',
       'distribution_channel_TA/TO', 'distribution_channel_Undefined',
       'reserved_room_type_A', 'reserved_room_type_B', 'reserved_r

In [ ]:
# remove binary cols which are less than 5% sparse
binary_cols = binary_cols[(df[binary_cols]>0).sum(0) > 0.05 * df.shape[0]]
print("no. of non-sparse binary_cols =", len(binary_cols))
binary_cols

no. of non-sparse binary_cols = 39


Index(['hotel_City Hotel', 'hotel_Resort Hotel', 'arrival_date_month_April',
       'arrival_date_month_August', 'arrival_date_month_December',
       'arrival_date_month_February', 'arrival_date_month_July',
       'arrival_date_month_June', 'arrival_date_month_March',
       'arrival_date_month_May', 'arrival_date_month_November',
       'arrival_date_month_October', 'arrival_date_month_September', 'meal_BB',
       'meal_HB', 'meal_SC', 'market_segment_Direct', 'market_segment_Groups',
       'market_segment_Offline TA/TO', 'market_segment_Online TA',
       'distribution_channel_Corporate', 'distribution_channel_Direct',
       'distribution_channel_TA/TO', 'reserved_room_type_A',
       'reserved_room_type_D', 'reserved_room_type_E', 'assigned_room_type_A',
       'assigned_room_type_D', 'assigned_room_type_E',
       'deposit_type_No Deposit', 'deposit_type_Non Refund',
       'customer_type_Transient', 'customer_type_Transient-Party',
       'reservation_status_Canceled', 'reser

In [ ]:
# removing sparse binary cols
non_binary_cols = df.columns[df.apply(pd.Series.nunique) > 2]
df = df[list(non_binary_cols) + list(binary_cols)]

print("df.shape =", df.shape)
df.head(4)

df.shape = (119390, 53)


,stays_in_weekend_nights,booking_changes,required_car_parking_spaces,previous_cancellations,arrival_date_day_of_month,adr,stays_in_week_nights,previous_bookings_not_canceled,days_in_waiting_list,total_of_special_requests,...,deposit_type_No Deposit,deposit_type_Non Refund,customer_type_Transient,customer_type_Transient-Party,reservation_status_Canceled,reservation_status_Check-Out,arrival_date_year_2015,arrival_date_year_2016,arrival_date_year_2017,is_repeated_guest_0
0,0,3,0,0,1,0.0,0,0,0,0,...,1,0,1,0,0,1,1,0,0,1
1,0,4,0,0,1,0.0,0,0,0,0,...,1,0,1,0,0,1,1,0,0,1
2,0,0,0,0,1,75.0,1,0,0,0,...,1,0,1,0,0,1,1,0,0,1
3,0,0,0,0,1,75.0,1,0,0,0,...,1,0,1,0,0,1,1,0,0,1


In [ ]:
# check if total cols = numeric_cols + binary_cols
assert (len(numeric_cols) + len(binary_cols) == df.shape[1])

print("cardinality of continuous columns:")
df[numeric_cols].apply(pd.Series.nunique)

cardinality of continuous columns:


stays_in_weekend_nights             17
booking_changes                     21
required_car_parking_spaces          5
previous_cancellations              15
arrival_date_day_of_month           31
adr                               8879
stays_in_week_nights                35
previous_bookings_not_canceled      73
days_in_waiting_list               128
total_of_special_requests            6
adults                              14
arrival_date_week_number            53
babies                               5
lead_time                          479
dtype: int64

In [ ]:
df["adr"].min(), df["adr"].max()

(-6.38, 5400.0)

In [ ]:
df.max()

stays_in_weekend_nights             19.0
booking_changes                     21.0
required_car_parking_spaces          8.0
previous_cancellations              26.0
arrival_date_day_of_month           31.0
adr                               5400.0
stays_in_week_nights                50.0
previous_bookings_not_canceled      72.0
days_in_waiting_list               391.0
total_of_special_requests            5.0
adults                              55.0
arrival_date_week_number            53.0
babies                              10.0
lead_time                          737.0
hotel_City Hotel                     1.0
hotel_Resort Hotel                   1.0
arrival_date_month_April             1.0
arrival_date_month_August            1.0
arrival_date_month_December          1.0
arrival_date_month_February          1.0
arrival_date_month_July              1.0
arrival_date_month_June              1.0
arrival_date_month_March             1.0
arrival_date_month_May               1.0
arrival_date_mon

### $Chi^2$ test to select important categorical features (i.e binary_cols here)

In [ ]:
X = df[binary_cols]

print("X.shape=", X.shape, "y.shape=", y.shape)

X.shape= (119390, 39) y.shape= (119390,)


In [ ]:
chi_scores = chi2(X, y)

feature_importnce = pd.DataFrame({
    "column": X.columns,
    "p_vals": chi_scores[1],         # p-value: (1-p_value) = Feature importance
    "chi2_scores": chi_scores[0],    # Chi^2 scores = Feature importance
    "is_correlated_to_label": chi_scores[1] < 0.05,    # 5% level of significance
}).sort_values(by=['p_vals'])

feature_importnce

,column,p_vals,chi2_scores,is_correlated_to_label
1,hotel_Resort Hotel,0.000000e+00,1478.773676,True
26,assigned_room_type_A,0.000000e+00,1762.821706,True
34,reservation_status_Check-Out,0.000000e+00,44224.000000,True
21,distribution_channel_Direct,0.000000e+00,2407.956141,True
33,reservation_status_Canceled,0.000000e+00,73114.503935,True
17,market_segment_Groups,0.000000e+00,4901.438975,True
16,market_segment_Direct,0.000000e+00,2545.319532,True
29,deposit_type_No Deposit,0.000000e+00,3368.650474,True
32,customer_type_Transient-Party,0.000000e+00,1452.584353,True
30,deposit_type_Non Refund,0.000000e+00,24293.423036,True


In [ ]:
print("previously len(binary_cols) =", len(binary_cols))

# new binary_cols = only the important binary_cols
binary_cols = list(feature_importnce.loc[feature_importnce.is_correlated_to_label == True]["column"].values)

print("now len(binary_cols) =", len(binary_cols))

previously len(binary_cols) = 39
now len(binary_cols) = 34


# 1-way ANOVA continuous valued column selection

In [ ]:
X = df[numeric_cols]

print("X.shape=", X.shape, "y.shape=", y.shape)

X.shape= (119390, 14) y.shape= (119390,)


In [ ]:
selector = SelectKBest(f_classif, k=1) # k does not matter here, hence set it to 1
selector.fit_transform(X, y)

feature_importnce = pd.DataFrame({
    "column": X.columns,
    "p_vals": selector.pvalues_,        # p-value: (1-p_value) = Feature importance
    "anova_scores": selector.scores_,   # ANOVA scores: is also Feature Importance
    "is_correlated_to_label": selector.pvalues_ < 0.05,    # 5% level of significance
}).sort_values(by=['p_vals'])

feature_importnce

,column,p_vals,anova_scores,is_correlated_to_label
1,booking_changes,0.000000e+00,2541.731401,True
2,required_car_parking_spaces,0.000000e+00,4744.260085,True
9,total_of_special_requests,0.000000e+00,6957.100859,True
13,lead_time,0.000000e+00,11222.198248,True
3,previous_cancellations,8.932657e-319,1465.864983,True
10,adults,1.078565e-95,431.598081,True
7,previous_bookings_not_canceled,1.489465e-87,394.072046,True
8,days_in_waiting_list,2.503568e-78,351.567768,True
5,adr,9.682934e-61,270.623533,True
12,babies,2.923024e-29,126.167626,True


In [ ]:
print("previously len(numeric_cols) =", len(numeric_cols))

# new binary_cols = only the important binary_cols
numeric_cols = list(feature_importnce.loc[feature_importnce.is_correlated_to_label == True]["column"].values)

print("now len(numeric_cols) =", len(numeric_cols))

previously len(numeric_cols) = 14
now len(numeric_cols) = 13


In [ ]:
print("previously df.shape=", df.shape, "y.shape=", y.shape)

# update df to keep only the important columns
df = df[numeric_cols + binary_cols]

print("now df.shape=", df.shape, "y.shape=", y.shape)

previously df.shape= (119390, 53) y.shape= (119390,)
now df.shape= (119390, 47) y.shape= (119390,)


# Model based selection
1.   fit a model typically Random-Forest and get feature importance.
2.   Select top k features

In [ ]:
# Demo clf.feature_importances_
rf = RandomForestClassifier(n_estimators=100).fit(df, y)
pred_y = rf.predict(df)
print("predictions =", pred_y[:30])

pd.DataFrame({
    "column": df.columns,
    "importance": rf.feature_importances_
}).sort_values(by=['importance'])

predictions = [0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]


,column,importance
9,babies,0.000022
44,arrival_date_month_December,0.000041
30,reserved_room_type_E,0.000049
34,arrival_date_month_June,0.000051
33,arrival_date_month_November,0.000057
45,arrival_date_month_October,0.000066
40,arrival_date_month_May,0.000066
32,arrival_date_month_March,0.000070
43,arrival_date_month_September,0.000077
36,arrival_date_month_April,0.000081


### OR use a feature-selector "SelectFromModel" with random-forest

In [ ]:
# fit a SelectFromModel with random-forest model
sfm = SelectFromModel(RandomForestClassifier(n_estimators=100), threshold="0.05*mean", max_features=30)
sfm.fit(df, y)

feature_importances = pd.DataFrame({
    "column": df.columns,
    "importance": sfm.estimator_.feature_importances_
}).sort_values(by=['importance'])

rf_columns = df.columns[(sfm.get_support())]
print("len(rf_columns) =", len(rf_columns))
df_rf = df[rf_columns]                         # OR  X_rf = sfm.transform(df) - returns numpy array

print("rf_columns =", rf_columns)
print("now df_rf.shape =", df_rf.shape, "y.shape=", y.shape)

feature_importances

len(rf_columns) = 19
rf_columns = Index(['booking_changes', 'required_car_parking_spaces',
       'total_of_special_requests', 'lead_time', 'previous_cancellations',
       'adr', 'assigned_room_type_A', 'reservation_status_Check-Out',
       'distribution_channel_Direct', 'reservation_status_Canceled',
       'market_segment_Groups', 'market_segment_Direct',
       'deposit_type_No Deposit', 'customer_type_Transient-Party',
       'deposit_type_Non Refund', 'distribution_channel_TA/TO',
       'customer_type_Transient', 'reserved_room_type_A',
       'market_segment_Offline TA/TO'],
      dtype='object')
now df_rf.shape = (119390, 19) y.shape= (119390,)


,column,importance
9,babies,0.000023
43,arrival_date_month_September,0.000050
7,days_in_waiting_list,0.000052
30,reserved_room_type_E,0.000055
44,arrival_date_month_December,0.000067
32,arrival_date_month_March,0.000069
33,arrival_date_month_November,0.000074
34,arrival_date_month_June,0.000074
36,arrival_date_month_April,0.000077
28,assigned_room_type_E,0.000078


In [ ]:
# OR you can directly call SelectFromModel().fit_transform(X, y):
df_rf = SelectFromModel(RandomForestClassifier(n_estimators = 100), threshold="0.05*mean", max_features=40).fit_transform(df, y)

print("now df_rf.shape =", df_rf.shape, "y.shape=", y.shape)

now df_rf.shape = (119390, 32) y.shape= (119390,)


# Modeling: Now use these features to train any classifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_rf, y, test_size=0.2, random_state=47)

In [ ]:
# fit model on training data
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

# inference/scoring: make predictions
y_pred = xgb.predict(X_test)
y_pred[:20]

array([1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0])

In [ ]:
# evaluate predictions
accuracy_score(y_test, y_pred)

1.0

In [ ]:
# accuracy_score() is same as:
sum(y_test == y_pred) / len(y_test)

1.0